# Non-Distributed LOTS Attack
As you might have noticed, there are two different ways to use the CLI. The first one is by passing the arguments directly to the script, and the second one is by using a configuration file. Using a configuration file is a good way to keep track of the experiments you are running, and it is also a good way to avoid passing the same arguments over and over again. This notebook will show you how to use a configuration file and how to pass the arguments directly to the script for the Non-Distributed LOTS Attack.

## Using a Configuration File


### Step 1: Create a Configuration File

As a first step, you need to have a default configuration file. You can create a configuration file by running the following command:
    
```sh
advsecurenet utils configs get -c lots_attack_config.yml -s -o ./default_lots_attack_config.yml
```

Here, we are also providing the `-o` flag to specify the output directory where the configuration file will be saved. You can also specify the output directory by passing the `-o` flag followed by the path to the directory where you want to save the configuration file. If you don't provide the `-o` flag, the configuration file will be saved in the current working directory with the name `lots_attack_config.yml`. If the file already exists, it will be overwritten. If the output path is a directory, the configuration file will be saved in that directory with the name `lots_attack_config.yml`.



In [1]:
!advsecurenet utils configs get -c lots_attack_config.yml -s -o ./default_lots_attack_config.yml

Saving default config to ./default_lots_attack_config.yml
Generated default configuration file lots_attack_config.yml!


In [2]:
# Let's check the content of the default config file
!advsecurenet utils configs get -c lots_attack_config.yml -p

**************************************************
Default configuration file for the lots_attack_config.yml:

{
    "model": {
        "model_name": "resnet18",
        "num_input_channels": 3,
        "num_classes": 10,
        "pretrained": true,
        "weights": "IMAGENET1K_V1",
        "is_external": false,
        "random_seed": null,
        "path_configs": {
            "model_arch_path": null,
            "model_weights_path": null
        },
        "norm_config": {
            "add_norm_layer": false,
            "norm_mean": [
                0.5,
                0.5,
                0.5
            ],
            "norm_std": [
                0.5,
                0.5,
                0.5
            ]
        }
    },
    "dataset": {
        "dataset_name": "cifar10",
        "download": true,
        "num_classes": 10,
        "dataset_part": "test",
        "train_dataset_path": null,
        "test_dataset_path": null,
        "random_sample_size": null,
        "prep

## Step 2: Modify the Configuration File

The default configuration file is capable of running the targeted `LOTS` attack on the `CIFAR10` dataset using the `ResNet18` model. You can modify the configuration file to run the LOTS attack on a different dataset or model. You can modify attack parameters, such as the epsilon value and step size. Additionally, you can select whether to use the GPU or CPU for the attack and decide whether to save the adversarial images

For the sake of this example, we will modify the configuration file to run the `LOTS` attack on the `CIFAR10` dataset using the `VGG16` model. We will also add a *normalization layer* to the model, use the *GPU* for the attack, and *save the adversarial images*. We won't use the whole dataset for the attack, but only the *10 random samples*. Finally, we will set the epsilon value to *0.1*, the attack mode to *single* which means that the attack will be performed only once, and we will let the package to automatically select the target classes and images.

You can find the updated configuration file [here](lots_attack_config.yml).


`LOTS` attack requires a deep feature layer to be used as a feature extractor. So, we need to decide which layer to use as a feature extractor. In this example, we will use the `VGG16` model. Let's find the layer names of the `VGG16` model by running the following command:

```sh
advsecurenet utils models layers -m VGG16 -n
```

Note that we are using the `-n` flag to include a normalization layer in the model as we have decided to add a normalization layer to the model we need to make sure that the normalization layer is included in the list of layers.

In [15]:
!advsecurenet utils models layers -m vgg16 -n

/Users/melih/miniconda3/envs/adv2/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/Users/melih/miniconda3/envs/adv2/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/Users/melih/miniconda3/envs/adv2/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/Users/melih/miniconda3/envs/adv2/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
Layers of vgg16:
Layer Name                    Layer Type                    
input                         NoneType                      
0.getattr                     NoneType                      
0.to                          None

We would like to use the `1.classifier.6` layer as the feature extractor. You can pick any layer from the list of layers provided by the command above.

Now, we need to update the configuration file with the layer name. You can find the updated configuration file [here](lots_attack_config.yml).

## Step 3: Run the Attack

Now that you have created and modified the configuration file, you can run the attack by using the following command:

```sh
advsecurenet attack lots -c lots_attack_config.yml
```

**Note:** Attack success rate is calculated as the percentage of adversarial images that are misclassified by the model and the benign images that are correctly classified by the model. Since the attack runs on a different dataset than the one used to train the model, the attack success rate might be lower than expected.

In [16]:
!advsecurenet attack lots -c lots_attack_config.yml

Files already downloaded and verified

Generating adversarial samples: 100%|██████████| 1/1 [00:15<00:00, 15.88s/batch]
                                                                                Attack Success Rate: 0.0000
Saving images: 100%|████████████████████████| 10/10 [00:00<00:00, 230.57image/s]
Attack completed successfully.


Congratulations! You have successfully run the Non-Distributed LOTS Attack using a configuration file. You can now use the configuration file to run the attack with different parameters without having to pass the arguments directly to the script.